# Initialize
This section is a developer aid; TODO remove later!
*Align with [storyboard Version 48](https://rubinobs.atlassian.net/wiki/pages/viewpage.action?pageId=132612364&pageVersion=48)*

If you are using this under Time Square and *get any errors* that you think come from this notebook, please tell Steve Pothier (steve.pothier@noirlab.edu).  Include the URL you used to get the error. 

In [1]:
# Parameters.
# Times Square replaces this cell with the user's parameters.
# So, don't put anything else here!

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
#!day_obs = '2024-09-25' # Value to use for local testing (Summit)
day_obs = "YESTERDAY"  # TODO Change to 'YESTERDAY' to test with default before push

# Total number of days of data to display (ending on day_obs)
number_of_days = "3"  # TODO Change to '1' to test with default before push

In [2]:
import datetime as dt
import os
from collections import defaultdict
from pprint import pformat, pp
from urllib.parse import urlencode, urlparse
import pandas as pd
from IPython.display import HTML

# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get("EXTERNAL_INSTANCE_URL"):
    dev_msg = (
        'Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch. \n'
        'TODO: Make the need for this go away by getting Logging_and_Reporting installed in RSP.'
    )
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype > /dev/null 2>&1
else:
    dev_msg = 'Imported lsst.ts.logging_and_reporting from local packages.'
from lsst.ts.logging_and_reporting.all_sources import AllSources
from lsst.ts.logging_and_reporting.all_reports import AllReports
import lsst.ts.logging_and_reporting.utils as ut
from lsst.ts.logging_and_reporting.reports import md, mdlist, mdpathlink
from lsst.ts.logging_and_reporting.reports import html_draft, html_beta

In [3]:
# Set default env to "usdf" and try before PUSH to repo.
# The default provided here is for local testing. Under Times Square it is ignored.
server = os.environ.get(
    "EXTERNAL_INSTANCE_URL", 
    ut.Server.usdf) # TODO try with "usdf" before push (else "summit")

In [4]:
# Normalize Parameters (both explicit Times Squares params, in implicit ones)
limit = 5000  # YAGNI for Auto get more if this isn't enough to get all requested DAYS
response_timeout = 3.05  # seconds, how long to wait for connection
read_timeout = 20  # seconds
timeout = (float(response_timeout), float(read_timeout))

date = ut.get_datetime_from_dayobs_str(day_obs)
# date:  is EXCLUSIVE (upto, but not including)
days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_date = date - dt.timedelta(days=days - 1)
max_date = date + dt.timedelta(days=1)
min_day_obs = min_date.strftime("%Y-%m-%d")  # Inclusive
max_day_obs = max_date.strftime("%Y-%m-%d")  # prep for Exclusive

# Experimental Features config
exp_tally = "false"  # default: true
exp_parsing = "true"  # default: true
experimental = dict()
experimental["exposure_type_tally"] = exp_tally == "true"
experimental["error_parsing"] = exp_parsing == "true"

In [5]:
# Read records from (almost) all sources
allsrc = AllSources(
    server_url=server,
    min_dayobs=min_day_obs,
    max_dayobs=max_day_obs,
    verbose=False,                # TODO change to False before push
    limit=limit,
    exclude_instruments = []      # TODO change to empty list before push
)
allrep = AllReports(allsrc=allsrc)

DEBUG add_exposure_flag instrument='LSSTComCamSim' set()
DEBUG add_exposure_flag instrument='LATISS' set()
DEBUG add_exposure_flag instrument='LSSTComCam' {'junk', 'questionable', 'unknown', 'good'}
DEBUG add_exposure_flag instrument='LSSTCam' {'unknown'}


In [6]:
instrum_str = ", ".join(list(allsrc.exp_src.instruments.keys()))
md(f"# Showing data for {min_date.date()} to {max_date.date()} for {instrum_str}")

# Showing data for 2024-11-05 to 2024-11-08 for LSSTComCamSim, LATISS, LSSTComCam, LSSTCam

# Table of Contents
* [Night Report](#Night-Report-BETA)
    - AuxTel
    - Simonyi
* [Almanac](#Almanac-BETA)
* [Summary plots of whole night](#Summary-plots-of-whole-night-DRAFT)
* [Time Accounting](#Time-Accounting-DRAFT)
* [Jira Tickets](#Jira-Tickets-BETA)
    - AuxTel
    - Simonyi
* [BLOCKS Observed](#BLOCKS-Observed-DRAFT)
    - AuxTel
    - Simonyi
* [Data Log](#Data-Log-DRAFT)
* [Narrative Log](#Narrative-Log-BETA)
* [Developer Only Section](#Developer-Only-Section-REMOVE)

## Night Report <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [7]:
# Night Report
# Display time log
allrep.nig_rep.time_log_as_markdown()

## Almanac <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [8]:
# Display various almanac values (for moon, sun)
#!rep.AlmanacReport().day_obs_report(min_day_obs)
allrep.alm_rep.day_obs_report()

**Almanac for the observing night starting 2024-11-05**

,(times in UTC),
Moon Rise,2024-11-05 12:26:23.360,
Moon Set,2024-11-05 02:21:05.324,
Moon Illumination,12%,(% illuminated)
Astronomical Twilight (morning),2024-11-05 08:16:31.085,(-18 degrees)
Astronomical Twilight (evening),2024-11-05 00:36:18.653,(-18 degrees)
Solar Midnight,2024-11-05 04:26:24.970,
Nautical Twilight (morning),2024-11-05 08:48:16.492,(-12 degrees)
Nautical Twilight (evening),2024-11-05 00:04:33.140,(-12 degrees)
Civil Twilight (morning),2024-11-05 09:18:46.299,(-6 degrees)
Civil Twilight (evening),2024-11-04 23:34:03.177,(-6 degrees)


## Summary plots of whole night <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

(content not yet defined in storyboard)

In [9]:
# Plot Observation (Exposure) gaps
# rollup,detail = allsrc.get_observation_gaps()
# allrep.plot_observation_gap_detail(detail)

## Time Accounting <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
TODO:
- Time-loss for Fault and Weather
- SlewTime from TMAEvent (or similar)

SlewTime (and probably others) from EFD are only EXPECTED (topic="lsst.sal.Scheduler.logevent_target"), not ACTUAL.
To get ACTUAL time, we need to use something like TMAEvent to combine multiple real events into a synthetic event.  See DM-46102.


In [10]:
# Exposure Tally
tally = await allsrc.night_tally_observation_gaps()
if tally:
    display(pd.DataFrame(tally))

,LSSTComCam,LSSTCam
Total Night,7:40:12,7:40:12
Total Exposure,4:28:06,7:30:13
Slew time(1),0:00:00,0:00:00
Readout time(2),0:23:36,1:13:54
Time loss to fault,NA,NA
Time loss to weather,NA,NA
Idle time,3:12:06,0:09:59
Number of exposures,616,1928
Mean readout time,0.000639,0.000639
Number of slews(1),0,0


- (1) *Expected* slew time as per Schedular
- (2) There is no practical way to get detector read-out time.  A value of 2.3 seconds was inferred from experiments.

## Jira Tickets <font style="background-color:green; color:white; font-size:20px">BETA</font> 
- Observing Operations - OBS
- TODO: better tickets using Jira API

In [11]:
if allsrc.urls:
    for url in allsrc.urls:
        md(f"- {mdpathlink(url)}")

- [OBS-648](https://rubinobs.atlassian.net/browse/OBS-648)

- [OBS-321](https://rubinobs.atlassian.net/browse/OBS-321)

- [OBS-646](https://rubinobs.atlassian.net/browse/OBS-646)

- [/rubinobs-lfa-cp/LOVE/OLE/2024/11/04/LOVE_OLE_2024-11-05T04:06:17.057.png](https://s3.cp.lsst.org/rubinobs-lfa-cp/LOVE/OLE/2024/11/04/LOVE_OLE_2024-11-05T04:06:17.057.png)

- [OBS-649](https://rubinobs.atlassian.net/browse/OBS-649)

- [obs-647](https://rubinobs.atlassian.net/browse/obs-647)

- [obs-321](https://rubinobs.atlassian.net/browse/obs-321)

- [obs-611](https://rubinobs.atlassian.net/browse/obs-611)

In [12]:
# Display Jira BLOCKS
front = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/"
tickets = allsrc.nig_src.nightly_tickets()
if tickets:
    mdstr = "#### Nightly Jira BLOCKs"
    for day, url_list in tickets.items():
        mdstr += f"\n- {day}"
        for ticket_url in url_list:
            mdstr += f'\n    - [{ticket_url.replace(front,"")}]({ticket_url})'
    md(mdstr)
else:
    endpoint_url = allsrc.nig_src.status["reports"]["endpoint_url"]
    md(f"No tickets found using: [API Data]({endpoint_url}) in `confluence_url`.")

No tickets found using: [API Data](https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241105&max_day_obs=20241108) in `confluence_url`.

## BLOCKS Observed <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
- TODO: BLOCKS observed

## Data Log <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

- NOTE: *modified all_sources.py:uniform_field_counts()*
- NOTE: Each tally count is now a link to a detailed table of exposures.

In [13]:
for instrum in allsrc.exp_src.exposures.keys():
    md(f'**{instrum}: {len(allsrc.exp_src.exposures[instrum])} total exposures**')
    df_dict = allsrc.fields_count_exposure(instrum)
    for field_name,df in df_dict.items():
        print(field_name.title().replace('_',' '))
        display(df.T)

**LSSTComCamSim: 0 total exposures**

Observation Type


""


Observation Reason


""


...

,good,questionable,junk,unknown,total
unknown,0,0,0,1,1
flat,0,0,0,656,656
bias,0,0,0,395,395
dark,0,0,0,876,876


Observation Reason


,good,questionable,junk,unknown,total
preimage,0,0,0,6,6
unknown,0,0,0,1,1
flat,0,0,0,1066,1066
bias,0,0,0,109,109
sflat_hi,0,0,0,50,50
dark,0,0,0,696,696


Science Program


,good,questionable,junk,unknown,total
E2184,0,0,0,10,10
E2236,0,0,0,100,100
unknown,0,0,0,334,334
E2180,0,0,0,3,3
E2136,0,0,0,201,201
E2229,0,0,0,24,24
E2230,0,0,0,201,201
E2234,0,0,0,1,1
E2183,0,0,0,3,3
E2016,0,0,0,448,448


In [14]:
# Exposure Report
if experimental.get("exposure_type_tally"): 
    #  md("### Experimental Exposure field Tally ENABLED")
    md("Tally exposures with specific values of selected fields")
else:
    md("### Experimental Exposure field Tally DISASBLED")
    
for instrum in allsrc.exp_src.exposures.keys():
    md(f'**{instrum}: {len(allsrc.exp_src.exposures[instrum])} total exposures**')
    if 0 == len(allsrc.exp_src.exposures[instrum]):
        continue

    # Facet counts
    if experimental.get("exposure_type_tally"): 
        counts,cc,f = allsrc.uniform_exposure_field_counts(instrum)
        for fname, tally in []: # counts.items():
            md(f"{fname}")
            df = pd.DataFrame.from_dict(tally, orient="index").T
            wdf = ut.wrap_dataframe_columns(df)
            display(HTML(wdf.to_html(render_links=True, escape=False)))

### Experimental Exposure field Tally DISASBLED

**LSSTComCamSim: 0 total exposures**

**LATISS: 0 total exposures**

**LSSTComCam: 616 total exposures**

**LSSTCam: 1928 total exposures**

Note: *Added counts of Exposures with messages*

## Narrative Log <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [15]:
# Narrative Report
# Time Log
allrep.nar_rep.time_log_as_markdown()

- **2024-11-05 10:06:04.409120** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Done for tonight. All TMA and CSCs are closed as requested.</pre>


- **2024-11-05 09:48:51.572165** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>2024 11 05 09:47:37.238 Hardpoint #2 hit high limit.</pre>


- **2024-11-05 09:46:08.288296** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>NO fault! Locking pin engaged and turning OFF PS. M1M3 lowered and will disabled.</pre>


- **2024-11-05 09:31:44.430342** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Retracted the mirror cover. Controlling with EUI again. Will move to Horizon...! Mirror will keep being raise per Petr's request.</pre>


- **2024-11-05 09:25:45.527290** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>We realized that we need retracted (open) for the mirror cover on Tuesday. Will enable the MTMount to retract the mirror.</pre>


...

- **2024-11-07 17:03:53.871075** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Jacques, Brian, Marina, Gonzalo and I had a quick chat and we all agreed to leave the set point today at 11ºC.</pre>


- **2024-11-07 16:07:41.295262** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>running BLOCK-T246 Taking bias images</pre>


- **2024-11-07 16:03:18.586752** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Rotator and Hexapods startup went well with no issues</pre>


- **2024-11-07 15:43:51.013520** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Gonzalo and Marina are now in the control room. Both are powering up the hexapods and the rotator. We might leave the warm-ups for later following Te-Wei's suggestion ([here|https://rubin-obs.slack.com/archives/C07QJMQ01FU/p1730994153557769]). 


Meanwhile, I am addressing the temperatures in preparation for tonight.</pre>


- **2024-11-07 14:46:42.064697** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>There was a fault on M1M3 just now and I recovered. 
It was associated with work inside the cell where Felipe D. closed the DAA438 valve while checking for leaks.</pre>


-----------
------------

# Developer Only Section <font style="background-color:red; olor:black; font-size:20px">REMOVE</font>
Contains stuff only expected to be useful to developers.

This may also contain sections that have moved out of the user section because they are no longer defined in the Storyboard.

## Overview 


In [16]:
# Display overview of Report context
try:
    import lsst.ts.logging_and_reporting.version

    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = "LIVE"

try:
    from lsst.summit.utils import ConsDbClient

    have_consdb = True
except:
    have_consdb = False
md(
    f"""## Project-Wide Night(s) Report 
- Run on logs and databases from **{server}/**
- Report **{days} observing night(s)** with the last reported night starting on **{date}**.
- min_dayobs={allsrc.min_dayobs!r}, max_dayobs={allsrc.max_dayobs!r}
- Using ***Prototype* Logging and Reporting** Version: **{lrversion}**
- {have_consdb = }
- {dev_msg}
""")

## Project-Wide Night(s) Report 
- Run on logs and databases from **https://usdf-rsp-dev.slac.stanford.edu/**
- Report **3 observing night(s)** with the last reported night starting on **2024-11-07 12:00:00**.
- min_dayobs='2024-11-05', max_dayobs='2024-11-08'
- Using ***Prototype* Logging and Reporting** Version: **0.1.dev145+gf2a5765.d20241014**
- have_consdb = True
- Imported lsst.ts.logging_and_reporting from local packages.


## Data Status
This may be useful for diagnosing reports that give unexpected results.

In [17]:
df = pd.DataFrame(allsrc.get_data_status())
table = df.to_html(
    index=False, 
    render_links=True,
    escape=False)
display(HTML(table))

Endpoint,Records,URL
nightreport/reports,0,https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241105&max_day_obs=20241108
exposurelog/instruments,3,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/instruments
exposurelog/exposures.LSSTComCamSim,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCamSim&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241105&max_day_obs=20241108
exposurelog/exposures.LATISS,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LATISS&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241105&max_day_obs=20241108
exposurelog/exposures.LSSTComCam,616,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCam&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241105&max_day_obs=20241108
exposurelog/exposures.LSSTCam,1928,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTCam&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241105&max_day_obs=20241108
exposurelog/messages,16,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/messages?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=2500&min_day_obs=20241105&max_day_obs=20241108
narrativelog/messages,402,https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-11-05T00%3A00%3A00&max_date_added=2024-11-08T00%3A00%3A00
efd/targets,0,NA


## This report uses the following data sources
- NightReport
- Exposurelog
- Narrativelog
- EFD
- ConsDB
- (DDV)
- (Almanac from Astroplan)

## DDV 

In [18]:
DDV = (
    f"{server}/rubintv-dev/ddv/index.html"
    if "summit" in server
    else f"{server}/rubintv/ddv/index.html"
)
md(f"Access DDV part of RubinTV: {DDV}")

Access DDV part of RubinTV: https://usdf-rsp-dev.slac.stanford.edu/rubintv/ddv/index.html

## Where was this run?

The environments that have everything needed to run this page are:

    https://summit-lsp.lsst.codes
    https://usdf-rsp-dev.slac.stanford.edu

However, Times Square does not run on the Summit. It does run on USDF-dev.


## Section overviews moved here

In [19]:
# Night Report Overview
allrep.nig_rep.overview()
# Exposure Report Overview
allrep.exp_rep.overview()
# Narrative Report Overview
allrep.nar_rep.overview()

### Overview for Service: `nightreport` [0]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241105&max_day_obs=20241108

None
- Got 0 records.  


### Overview for Service: `exposurelog` [16]

...

None
- Got 16 records.  


### Overview for Service: `narrativelog` [402]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-11-05T00%3A00%3A00&max_date_added=2024-11-08T00%3A00%3A00

None
- Got 402 records.  


In [20]:
# Conditionally display our current ability to connect to all needed endpoints.
if False and not os.environ.get("EXTERNAL_INSTANCE_URL"):
    md("## Dashboard")
    md("(This is not done when running under Times Square.)")
    %run ./dashboard.ipynb

### According to Storybaord, this section should be removed ...
... and replaced with a simple table tally of Good, Questionable, and  Junk exposures per instrument.
| Symbol | Meaning |
|:---|:---|
| <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>| Good |
| <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font> | Questionable |
| <font style="background-color:red; color:black; font-size:20px">&nbsp;R&nbsp;</font> | Junk |


In [21]:
# Time Log
allrep.exp_rep.time_log_as_markdown()

### Instrument: LSSTComCam (16)

* CC_O_20241105_000019 : 2024-11-06T00:05:06.599261
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`We suddenly got out of focus ! the hexapods moved ! FocusZ changed from 0.22 to 0.44`

* CC_O_20241105_000026 : 2024-11-06T00:12:34.276724
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`Hexapods moved again and the focus deteriorated !`

* CC_O_20241105_000033 : 2024-11-06T00:38:34.849868
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`BLOCK-88`

* CC_O_20241105_000034 : 2024-11-06T00:38:34.811449
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`BLOCK-88`

...

* CC_O_20241106_000044 : 2024-11-07T01:29:42.989914
    - <font style="background-color:red; color:black; font-size:20px">&nbsp;R&nbsp;</font>`The best one. Step 9 for BLOCK-T92.`

* CC_O_20241106_000045 : 2024-11-07T01:31:23.489238
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`Looks good, donuts`

* CC_O_20241106_000078 : 2024-11-07T02:08:04.439645
    - <font style="background-color:red; color:black; font-size:20px">&nbsp;R&nbsp;</font>`Looks good, donuts`

* CC_O_20241106_000163 : 2024-11-07T04:41:14.472781
    - <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font>`Looks good, donuts`

* CC_O_20241106_000178 : 2024-11-07T05:01:19.057283
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`Lost tracking during the exposure. Streaks`

## Finale

In [22]:
print(f"Finished {str(dt.datetime.now())}")

Finished 2024-11-08 23:27:38.490611
